### Wiener process and Geometric Brownian Motion (GBM)

In [380]:
from math import sqrt, exp
import numpy as np

In [381]:
def rnorm(mu=0, sigma=1):
    return np.random.normal(0, 1)


def wiener_process(T):
    wiener = np.zeros(T)
    for t in range(T):
        Wt = sqrt(t + 1) * rnorm(0, 1)
        wiener[t] = Wt
        
    return wiener


def GBM(S0, mu, sigma, T):
    gbm = np.zeros(T + 1)
    gbm[0] = S0
    wiener = wiener_process(T)
    for t in range(1, T + 1):
        drift = mu - (sigma ** 2) / 2
        drift = drift * t
        vol = sigma * wiener[t - 1]
        St = S0 * exp(drift + vol)
        gbm[t] = St
    
    return gbm

#### Geometric Brownian Motion (GBM) moments
$\mathrm{E}[S_t] = S_0 \cdot e^{\mu t}, \, t=1,\ldots, T$

$\mathrm{Var}[S_t] = S_0^2 \cdot e^{2\mu t} \cdot (e^{\sigma^2 t} - 1), \, t=1,\ldots, T$

In [422]:
def generate_sample_with_offset(S0, mu, sigma, T, sample_size):
    sample = np.zeros(shape=(sample_size, T + 1))
    for i in range(sample_size): 
        gbm = GBM(S0, mu, sigma, T + 1)
        sample[i] = gbm[1:T + 2]
        
    return sample


def generate_sample(S0, mu, sigma, T, sample_size):
    sample = np.zeros(shape=(sample_size, T + 1))
    for i in range(sample_size): 
        gbm = GBM(S0, mu, sigma, T)
        sample[i] = gbm
        
    return sample


def gbm_mean(S0, mu, t):
    return S0 * exp(mu * t)


def gbm_std(S0, mu, sigma, t):
    std = S0 ** 2
    std *= exp(2 * mu * t)
    std *= (exp(sigma ** 2 * t) - 1)
    
    return sqrt(std)

In [456]:
sample_size = 1000
S = 10
T = 10

##### Test case 1
For $\mu = 0$ we expect $\mathrm{E}[S_t] = S_0, \, t=1,\ldots, T$

In [457]:
S0 = 40
mu = 0
sigma = 0.25

sample = generate_sample(S0, mu, sigma, T, sample_size)
sample.mean(axis=0)

array([40.        , 39.79991249, 39.52958531, 38.89862582, 40.06080848,
       39.01409927, 39.97001787, 39.86729242, 40.48300978, 39.45133916,
       41.24893523])

##### Test case 2

For $\mu > 0$ we expect $\mathbb{E}[S_t] = S_0 \cdot e^{\mu t}, \, t=1,\ldots, T$

In [458]:
S0 = 40
mu = 0.1
sigma = 0.25

sample = generate_sample(S0, mu, sigma, T, sample_size)
sample.mean(axis=0)

array([ 40.        ,  44.32607478,  47.85027293,  53.80183427,
        57.9746093 ,  64.4056902 ,  73.79651986,  81.45202112,
        86.64733411,  98.58983373, 108.57619567])

In [459]:
for t in range(0, T + 1):
    print(gbm_mean(S0, mu, t))

40.0
44.20683672302591
48.85611032640679
53.99435230304013
59.672987905650814
65.94885082800513
72.88475201562036
80.55010829881907
89.02163713969871
98.384124446278
108.7312731383618


### Shortest path algorithms

In [460]:
import networkx as nx
import itertools
import matplotlib.pyplot as plt

In [461]:
class ShortestPathResult():
    def __init__(self):
        self.dist = {}
        self.prev = {}

#### Dijkstra - basic implementation

In [462]:
def reconstruct_shortest_path(source, target, prev):
    if source != target:
        path = reconstruct_shortest_path(source, prev[target], prev)
        path.append(target)
        return path
    else:
        path = [source]
        return path

def dijkstra(G, source, target, weight='weight'):
    memo = ShortestPathResult()
    memo.prev = {node: None for node in G.nodes}
    memo.dist = {node: np.Inf for node in G.nodes}
    memo.dist[source] = 0
    memo.unvisited = memo.dist.copy()
    current = source
    while (len(memo.unvisited) > 0) & (current != target):
        for nbr, attrs in G[current].items():
            if not nbr in memo.unvisited:
                continue
            d = attrs[weight] 
            new_dist = memo.dist[current] + d
            if new_dist < memo.dist[nbr]:
                memo.dist[nbr] = new_dist
                memo.prev[nbr] = current
   
        del memo.unvisited[current]
        current = min(memo.unvisited, key=memo.unvisited.get)    
    shortest_path = reconstruct_shortest_path(source, target, memo.prev)
    
    return shortest_path, memo.dist[target]

#### Dynamic programming

##### Top-down approach

We will use the fact that $SP(s, t) = \min\{SP(s, u) + w(u, t) | (u, t) \in E\}$

First top-down approach is implemented. I.e., we start from target node and recourse until we end up at the source.

Requirements:
* Graph should be directed, i.e., need to support innode operation

In [463]:
def sp_dp_recurse(G, target, memo, weight='weight'):
    if target in memo.dist:
        return memo.dist[target]
    memo.dist[target] = np.Inf
    memo.prev[target] = None
    for node in G.predecessors(target):
        new_dist = sp_dp_recurse(G, node, memo) + G[node][target][weight]
        if new_dist < memo.dist[target]:
            memo.dist[target] = new_dist
            memo.prev[target] = node
            
    return memo.dist[target]
    

def shortest_path_dp(G, source, target, weight='weight'):
    memo = ShortestPathResult()
    memo.dist[source] = 0 # base state
    memo.prev[source] = None
    dist = sp_dp_recurse(G, target, memo)
    shortest_path = reconstruct_shortest_path(source, target, memo.prev)
    
    return shortest_path, memo.dist[target]

##### Bottom-up approach

We also implement bottom-up approach. I.e., we start at source at build solution until we reach the target.

This imposes an additional requirement:
* Graph should be [topologically sorted](https://en.wikipedia.org/wiki/Topological_sorting#:~:text=In%20computer%20science%2C%20a%20topological,before%20v%20in%20the%20ordering.) - linear ordering of its vertices such that for every directed edge uv from node u to node v, u comes before v in the ordering

Actually, due to the structure of our graph, topological sort is not needed. The easiest is to start from source and keep track in deque which node should be visited next.
We iterate until deque is empty or we reach target node.

In [464]:
# TRY TO REWRITE THIS!
# Understand why it is 10x slower than other implementations. Probably because of queue

from collections import deque

def shortest_path_dp_bottomup(G, source, target, weight='weight'):
#    G = nx.topological_sort(G)
    memo = ShortestPathResult()
    memo.prev = {node: None for node in G.nodes}
    memo.dist = {node: np.Inf for node in G.nodes}
    memo.dist[source] = 0
    current = source
    q = deque()
    q.append(source)
    while (len(q) > 0) & (current != target):
        for nbr in G.successors(current):
            q.append(nbr)
            new_dist = memo.dist[current] + G[current][nbr][weight]
            if new_dist < memo.dist[nbr]:
                memo.dist[nbr] = new_dist
                memo.prev[nbr] = current
        current = q.popleft()    
    shortest_path = reconstruct_shortest_path(source, target, memo.prev)
    
    return shortest_path, memo.dist[target]

#### Test case

In [465]:
ss = list(itertools.product(range(T + 1), range(S + 1)))
nodes = [f'S_{stage}_{state}' for stage, state in ss]
edges = []
for stage, state in ss:
    if stage == T:
        continue
    for i in range(5):
        next_state = state + i
        if S < next_state:
            continue
        weight = np.random.normal(loc=10, scale=2)
        edge = (f'S_{stage}_{state}', f'S_{stage + 1}_{next_state}', {'weight':weight})
        edges.append(edge)
        
G = nx.DiGraph(edges)
"""
G = nx.DiGraph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
"""

'\nG = nx.DiGraph()\nG.add_nodes_from(nodes)\nG.add_edges_from(edges)\n'

In [466]:
path1 = nx.shortest_path(G, 'S_0_0', 'S_10_10', weight='weight')
path2 = dijkstra(G, 'S_0_0', 'S_10_10')[0]
path1 == path2

True

In [467]:
dist1 = dijkstra(G, 'S_0_0', 'S_10_10')[1]
dist2 = shortest_path_dp(G, 'S_0_0', 'S_10_10')[1]
dist3 = shortest_path_dp_bottomup(G, 'S_0_0', 'S_10_10')[1]
dist1 == dist2 == dist3

True

In [468]:
%%timeit
nx.shortest_path(G, 'S_0_0', 'S_10_10', weight='weight')

230 µs ± 2.96 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [469]:
%%timeit
shortest_path_dp(G, 'S_0_0', 'S_10_10')

489 µs ± 8.39 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [470]:
%%timeit
dijkstra(G, 'S_0_0', 'S_10_10')

824 µs ± 53.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [471]:
%%timeit
shortest_path_dp_bottomup(G, 'S_0_0', 'S_10_10')

808 ms ± 4.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


##### Conclusions

1. Networkx package has very well optimized shortest path algorithms (Dijkstra by default)
2. Dynamic programming top-down approach seems to efficient for small networks
3. Dynamic programming bottom-up (to me the most inuitive approach) is an order of magnitude slower than other algorithms 

### Buses electric fleet problem
**write down what we are trying to solve - DON'T FORGET**

In [472]:
def operating_cost(bias, slope, random_realization):
    return bias + slope * random_realization


def investment_cost(scale, random_realization1, random_realization2):
    return scale * random_realization1 * random_realization2


def generate_graph(investment_tech_obs, investment_rate_obs, diesel_obs, electric_obs):
    ss = list(itertools.product(range(T + 1), range(S + 1)))
    edges = []
    for stage, state in ss:
        if stage == T:
            investment = investment_cost(investment_scale, investment_tech_obs[stage], investment_rate_obs[stage])
            cost = (S - state) * investment * annual_discount[stage]
            edge = (f'S_{stage}_{state}', f'S_{T + 1}_{S}', {'weight':cost})
            edges.append(edge)
        else:
            for i in range(yearly_max_buses + 1):
                next_state = state + i
                if S < next_state:
                    continue
                investment = investment_cost(investment_scale, investment_tech_obs[stage], investment_rate_obs[stage])
                operating_diesel = operating_cost(diesel_bias, diesel_slope, diesel_obs[stage])
                operating_electric = operating_cost(electric_bias, electric_slope, electric_obs[stage])
                cost = next_state * operating_electric
                cost += (S - next_state) * operating_diesel
                cost += i * investment
                cost *= annual_discount[stage]
                edge = (f'S_{stage}_{state}', f'S_{stage + 1}_{next_state}', {'weight':cost})
                edges.append(edge)

    return nx.DiGraph(edges)


def descriptive_statistics(np_array):
    d = {}
    d['min'] = np.min(np_array)
    d['median'] = np.median(np_array)
    d['mean'] = np.mean(np_array)
    d['max'] = np.max(np_array)
    
    return d

#### Perfect Information

In [473]:
discount_factor = 0.98

diesel_s0 = 40
diesel_mu = 0
diesel_sigma = 0.25

electric_s0 = 150
electric_mu = 0
electric_sigma = 0.25

investment_tech_s0 = 1
investment_tech_mu = -0.05
investment_tech_sigma = 0.25

investment_rate_s0 = 1
investment_rate_mu = 0
investment_rate_sigma = 0.25

In [474]:
random_diesel = generate_sample(diesel_s0, diesel_mu, diesel_sigma, T, sample_size)
random_electric = generate_sample(electric_s0, electric_mu, electric_sigma, T, sample_size)
random_investment_tech = generate_sample(investment_tech_s0, investment_tech_mu, investment_tech_sigma, T, sample_size)
random_investment_rate = generate_sample(investment_rate_s0, investment_rate_mu, investment_rate_sigma, T, sample_size)
annual_discount = np.full(T + 1, discount_factor)
annual_discount = np.cumprod(annual_discount)

In [475]:
investment_scale = 4 * 10 ** 5
diesel_bias = 3600
diesel_slope = 155
electric_bias = 0
electric_slope = 115
yearly_max_buses = 4
pi_paths = []
pi_costs = []
for i in range(sample_size):
    G = generate_graph(random_investment_tech[i], random_investment_rate[i], random_diesel[i], random_electric[i])
    
    path, cost = shortest_path_dp(G, 'S_0_0', f'S_{T + 1}_{S}')
    pi_paths.append(path)
    pi_costs.append(cost)

In [476]:
pi_costs = np.round(pi_costs)

In [477]:
descriptive_statistics(pi_costs)

{'min': 777891.0, 'median': 1589541.5, 'mean': 1643184.751, 'max': 3798830.0}

#### Static Policy

In [478]:
expected_diesel = np.full(T + 1, 40)
expected_electric = np.full(T + 1, 150)
expected_investment_tech = np.zeros(T + 1)
expected_investment_rate = np.full(T + 1, 1)
for t in range(T + 1):
    expected_investment_tech[t] = np.exp(-0.05 * t)

G = generate_graph(expected_investment_tech, expected_investment_rate, expected_diesel, expected_electric)
si_path, si_cost = shortest_path_dp(G, 'S_0_0', f'S_{T + 1}_{S}')

In [479]:
round(si_cost)

2821089.0

In [480]:
si_cost

2821088.8390174466

#### Value of knowing perfect information

Birge introduces the concept as follows:
1. Having perfect information is the best. We can calculate average cost given such information, WS — wait-and-see
2. Knowing that uncertainty exists and using it during the optimization is second best, its average cost is RP — recourse problem
3. Taking uncertainty average and pretending it doesn't exist always yields the worst solution. Given static policy we can calculate EEV — expectation of expected value, i.e., calculate the cost of expected value solution given a realization of random process. If the process is finite discrete, we can calculate expectation exactly, otherwise, approximate it with a mean over big enough sample
We always have $WS \leq RP \leq EEV$

$EVPI = RP - WS$, expexted value of perfect information is the difference between solution knowing perfect information and solving stochastic problem
$VSS = EEV - RP$, value of stochastic solution is the difference between solving stochastic problem and expected value solution

In our case, we have EEV and WS and we would like to calculated a gap between the two, i.e., $gap = WS - EEV$ 

In [481]:
def eev_ws_gap(si_costs, pi_costs):
    return round(np.mean(si_costs) - np.mean(pi_costs))


def get_path_cost(path, investment_tech_obs, investment_rate_obs, diesel_obs, electric_obs):
    total_cost = 0
    for source, target in zip(path[:-1], path[1:]):
        source_stage = int(source.split('_')[-2])
        source_state = int(source.split('_')[-1])
        target_stage = int(target.split('_')[-2])
        target_state = int(target.split('_')[-1])
        if target_stage == T + 1:
            investment = investment_cost(investment_scale, investment_tech_obs[source_stage], investment_rate_obs[source_stage])
            cost = (S - source_state) * investment * annual_discount[source_stage]        
        else:
            investment = investment_cost(investment_scale, investment_tech_obs[source_stage], investment_rate_obs[source_stage])
            operating_diesel = operating_cost(diesel_bias, diesel_slope, diesel_obs[source_stage])
            operating_electric = operating_cost(electric_bias, electric_slope, electric_obs[source_stage])
            cost = target_state * operating_electric
            cost += (S - target_state) * operating_diesel
            cost += (target_state - source_state) * investment
            cost *= annual_discount[source_stage]
        total_cost += cost
        
    return total_cost

##### Test

Comparing perfect information costs found by shortest path algorithm to the ones calculated given path  

In [482]:
for i in range(10):
    c = get_path_cost(pi_paths[i], random_investment_tech[i], random_investment_rate[i], random_diesel[i], random_electric[i])
    print(pi_costs[i] == round(c))

True
True
True
True
True
True
True
True
True
True


### Gaps between perfect information and static policy

##### Mean of gaps between perfect information and static policy costs

In [490]:
si_costs = []
for i in range(sample_size):
    c = get_path_cost(si_path, random_investment_tech[i], random_investment_rate[i],random_diesel[i], random_electric[i])
    si_costs.append(c)
si_costs = np.round(si_costs)
gaps = si_costs - pi_costs
print(f'Gaps mean: {np.mean(gaps)}')
print(f'Gaps std: {np.std(gaps)}')

Gaps mean: 1200490.519
Gaps std: 2731173.860312062


##### Alternatively, gap between perfect information and expected value solution means
This yields the same results as above, but is more in the spirit of the approach

In [491]:
eev_ws_gap(si_costs, pi_costs)

1200491.0

In [492]:
np.mean(si_costs)

2843675.27

In [493]:
np.mean(pi_costs)

1643184.751

In [509]:
np.std(pi_costs) / (np.sqrt(sample_size) * np.mean(pi_costs))

0.008332333476405654

In [494]:
5 * np.std(si_costs) / np.sqrt(sample_size)

452029.0183517485

In [495]:
5 * np.std(pi_costs) / np.sqrt(sample_size)

68457.81654338294

Test - perfect information is a lower bound of static policy, i.e., it should never be higher than static policy cost

In [496]:
sum(pi_costs > si_costs) == 0

True

### Forcast-based reoptimization (FRM)

**Motivation**

Imagine that a flight has a pre-defined route based on the weather forecast week in advance which is updated hourly. Initially, flight might follow given path, but as soon as hourly update is given, we are able to re-optimize path. We will use observation for the next hour, while for the rest we might take an average.

Similar, we might know a diesel price for the next year, while for the rest we can use an average.

PITANJE:
Uzimamo li stara ocekivanja procesa ili se u svakom stage-u pravimo kao da tek krecemo?

In [498]:
def fill_observation_with_expected_value(obs, current_stage, mu):
    for i in range(current_stage + 1, len(obs)):
        obs[i] = gbm_mean(obs[current_stage], mu, i - current_stage)
        
    return obs
        

def frm(source, target, investment_tech_obs, investment_rate_obs, diesel_obs, electric_obs):
    frm_path = []
    frm_path.append(source)
    for i in range(T + 1):
        it = fill_observation_with_expected_value(investment_tech_obs.copy(), i, investment_tech_mu)
        ir = fill_observation_with_expected_value(investment_rate_obs.copy(), i, investment_rate_mu)
        di = fill_observation_with_expected_value(diesel_obs.copy(), i, diesel_mu)
        el = fill_observation_with_expected_value(electric_obs.copy(), i, electric_mu)
        G = generate_graph(it, ir, di, el)
        sp, _ = shortest_path_dp(G, source=frm_path[-1], target=target)
        frm_path.append(sp[1])
    frm_cost = get_path_cost(frm_path, investment_tech_obs, investment_rate_obs, diesel_obs, electric_obs)
    
    return frm_path, frm_cost

In [499]:
frm_paths = []
frm_costs = []
for i in range(sample_size):
    p, cst = frm('S_0_0', f'S_{T + 1}_{S}', random_investment_tech[i], random_investment_rate[i], random_diesel[i], random_electric[i])
    frm_paths.append(p)
    frm_costs.append(cst)

In [500]:
np.mean(frm_costs)

2706933.05632744

#### Gap between perfect information and forecast-based re-optimization

In [502]:
frm_costs = []
for i in range(sample_size):
    c = get_path_cost(frm_paths[i], random_investment_tech[i], random_investment_rate[i],random_diesel[i], random_electric[i])
    frm_costs.append(c)
frm_costs = np.round(frm_costs)
gaps = frm_costs - pi_costs
eev_ws_gap(frm_costs, pi_costs)

1063748.0

In [504]:
np.mean(frm_costs)

2706933.07

In [503]:
5 * np.std(frm_costs) / np.sqrt(sample_size)

410777.012822252

### Gurobi optimization

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import math

In [ ]:
m = gp.Model('so')
at = [m.addVar(vtype=GRB.INTEGER, name=f'a_{t}') for t in range(T)]
st = [m.addVar(vtype=GRB.INTEGER, name=f's_{t}') for t in range(T + 1)]

m.update()

In [ ]:
electric = [150 * 115 * (st[i] + at[i]) for i in range(T)]
diesel = [(3600 + 155 * 40) * (10 - st[i] - at[i]) for i in range(T)]
invest = [4*10**5 * math.exp(-0.05*t) * at[i] for i in range(T)]
final = [(10 - st[T]) * (4 * 10 ** 5) * math.exp(-0.05 * T)]

In [ ]:
obj = []
obj.extend(electric)
obj.extend(diesel)
obj.extend(invest)
obj.extend(final)

m.setObjective(sum(obj))

In [ ]:
try:

    # Create a new model
    m = gp.Model("mip1")

    # Create variables
    x = m.addVar(vtype=GRB.BINARY, name="x")
    y = m.addVar(vtype=GRB.BINARY, name="y")
    z = m.addVar(vtype=GRB.BINARY, name="z")

    # Set objective
    m.setObjective(x + y + 2 * z, GRB.MAXIMIZE)

    # Add constraint: x + 2 y + 3 z <= 4
    m.addConstr(x + 2 * y + 3 * z <= 4, "c0")

    # Add constraint: x + y >= 1
    m.addConstr(x + y >= 1, "c1")

    # Optimize model
    m.optimize()

    for v in m.getVars():
        print('%s %g' % (v.varName, v.x))

    print('Obj: %g' % m.objVal)

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')